# imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as st
from sympy import *
import time as dt
import warnings
from IPython.display import Math,Latex
from pandas.errors import SettingWithCopyWarning

# sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures,StandardScaler,MinMaxScaler
from sklearn.decomposition import PCA

from statsmodels.api import *

from scipy.stats import jarque_bera, kstest, kstwobign

# Задача 3.1. 

Для оценки удовлетворенности населения жизнью проведен социологический опрос в 80 странах мира. В таблице представлены агрегированные результаты проведенного опроса в разрезе стран, единицей измерения показателей является доля (%) респондентов, положительно оценивающих параметр. В качестве основных параметров удовлетворенности жизнью выбраны следующие:
- х1 – работа, %;
- х2 – здоровье, %; 
- х3 - материальное благополучие, %; 
- х4 - достижение поставленных целей, %; 
- х5 - социальный статус, %; 
- х6 - социальные контакты, %. 
<br>

Необходимо с помощью метода главных компонент выявить факторы удовлетворенности населения жизнью, ранжировать страны по уровню удовлетворенности населения жизнью.  

## Возьмем данные из `Исходные данные для задачи 3.1.pdf`

In [2]:
def get_countries_back(text:str)->str:
    repladict = {
        'Буркина-фасо':'Буркина-Фасо',
        'республика': ' Республика',
        'Тринидадитобаго':'Тринидад и Тобаго',
        'Боснияигерцеговина':'Босния и Герцеговина',
        'Сьерралеон':'Сьерра Леон',
        '-африканская': '-Африканская',
        
    }
    text = text.capitalize()
    for key,value in repladict.items():
        text = text.replace(key,value)
        
    return text

# Скопируем таблицу из Исходные данные для задачи 3.1.pdf
text3_1 ='''№ Страна х1 х2 х3 х4 х5 х6 1 Израиль 80 80 71 88 81 85 2 Греция 80 82 57 90 92 79 3 Словакия 76 72 47 85 78 93 4 Эстония 79 64 46 72 79 85 5 Венгрия 83 69 43 88 88 90 6 Португалия 90 80 47 92 93 87 7 Польша 82 72 67 87 91 89 8 Чили 81 73 68 90 93 83 9 Латвия 79 63 33 79 80 78 10 Хорватия 78 77 48 83 74 90 11 Болгария 73 67 29 77 77 81 12 Тринидад и Тобаго 76 82 40 97 93 85 13 Сербия 73 73 35 84 77 82 14 Белоруссия 66 55 34 70 71 88 15 Перу 74 72 54 96 89 79 16 Албания 72 75 43 78 68 79 17 Россия 74 56 36 79 83 88 18 Казахстан 82 68 51 88 81 88 19 Азербайджан 73 68 42 87 79 72 20 Босния и Герцеговина 76 75 39 80 67 74 21 Украина 71 55 23 74 78 81 22 Иран 71 82 55 87 81 62 23 Югославия 71 82 34 93 81 78 24 Грузия 63 50 22 86 83 54 25 Армения 61 53 31 93 89 67 26 Эквадор 80 76 57 98 93 78 27 Гайана 79 83 69 90 75 83 28 Ямайка 82 88 50 98 80 91 29 Турция 71 76 44 85 68 64 30 Доминиканская Республика 69 80 57 96 92 84 31 Сальвадор 82 80 60 97 89 72 32 Шри-Ланка  86 77 58 91 76 82 33 Таиланд  91 79 63 95 75 82 34 Боливия 83 79 67 94 90 82 35 Парагвай 85 84 63 93 96 89 36 Филиппины 83 77 68 96 94 77 37 Ботсвана 58 67 41 92 83 83 38 Молдова 68 60 39 79 73 83 39 Гана  84 87 64 95 77 84 40 Намибия 84 87 61 98 86 83 41 Гондурас 84 83 65 95 91 81 42 Индонезия 63 83 62 95 92 78 43 Киргизия 78 74 48 91 86 85 44 ЮАР 66 79 42 97 83 88 45 Таджикистан 78 75 69 91 76 65 46 Вьетнам 72 79 59 98 92 79 47 Марокко  69 88 71 90 89 85 48 Никарагуа  80 80 62 98 91 83 49 Индия 74 85 61 91 72 66 50 Камбоджи  80 69 51 81 87 82 51 Кения  57 70 25 98 78 79 52 Бангладеш  76 73 63 94 87 53 53 Гана  54 66 34 98 88 63 54 Камерун 63 69 40 93 85 73 55 Йемен 74 80 53 88 84 75 56 Мадагаскар  46 76 24 96 77 77 57 Мавритания  57 79 47 93 85 81 58 Нигерия 65 80 40 92 81 72 59 Уганда 53 64 35 96 79 85 60 Сенегал  39 68 27 89 85 81 61 Гаити  51 51 35 81 66 64 62 Ангола  72 67 54 90 83 58 63 Танзания  45 67 21 95 74 76 64 Замбия  48 78 34 93 83 62 65 Руанда  41 64 37 88 77 56 66 Малави  62 77 64 99 88 72 67 Судан 65 77 64 97 89 89 68 Афганистан  71 79 53 83 64 54 69 Гвинея  68 75 27 96 86 58 70 Эфиопия  50 79 33 89 74 76 71 СьерраЛеон  49 47 19 98 81 59 72 Центрально-Африканская  республика 78 81 31 96 74 56 73 Мали 30 71 30 99 86 75 74 Буркина-Фасо 46 70 27 94 83 73 75 Либерия 47 70 46 100 82 58 76 Чад 78 69 52 93 79 57 77 Мозамбик 74 82 46 93 89 75 78 Нигер 54 82 52 99 93 77 79 Конго 60 74 40 98 79 67 80 Зимбабве 49 72 27 91 81 81'''

# Преобразцем полученную строку в массив так, чтобы страна была одним элементом массива
strs3_1 = np.array(text3_1.lower().replace(' и ','и').replace('  ',' ').replace(' республика','Республика').split(' '))

# Изменим размерность массива и запишем данные в pandas.DataFrame
df3_1 = pd.DataFrame(strs3_1.reshape(strs3_1.size//8,8))

# Обозначим первую строку как заголовок, первый столбец за индекс и выполним обратное преобразование названий стран
df3_1.columns = df3_1.iloc[0]
df3_1 = df3_1.drop(0).reset_index(drop=True)
df3_1.set_index('№', inplace=True)
df3_1['страна'] = df3_1['страна'].apply(get_countries_back)

# Переименуем столбцы в удобный вид
df3_1.columns = ['Страна']+[f'x{i}' for i in range(1,7)]

# Трансформируем значения числовых столбцов в числовой тип данных
for i in range(1,7):
    df3_1[f'x{i}']=df3_1[f'x{i}'].transform(int)


df3_1

,Страна,x1,x2,x3,x4,x5,x6
№,,,,,,,
1,Израиль,80,80,71,88,81,85
2,Греция,80,82,57,90,92,79
3,Словакия,76,72,47,85,78,93
4,Эстония,79,64,46,72,79,85
5,Венгрия,83,69,43,88,88,90
...,...,...,...,...,...,...,...
76,Чад,78,69,52,93,79,57
77,Мозамбик,74,82,46,93,89,75
78,Нигер,54,82,52,99,93,77


## Перейдем к выполнению алгоритма главных компонент

### 1. Центрирование данных

In [3]:
data_centered = df3_1.iloc[:,1:] - np.mean(df3_1.iloc[:,1:], axis=0)
data_centered

,x1,x2,x3,x4,x5,x6
№,,,,,,
1,10.8125,6.5375,24.425,-2.7,-1.4,8.65
2,10.8125,8.5375,10.425,-0.7,9.6,2.65
3,6.8125,-1.4625,0.425,-5.7,-4.4,16.65
4,9.8125,-9.4625,-0.575,-18.7,-3.4,8.65
5,13.8125,-4.4625,-3.575,-2.7,5.6,13.65
...,...,...,...,...,...,...
76,8.8125,-4.4625,5.425,2.3,-3.4,-19.35
77,4.8125,8.5375,-0.575,2.3,6.6,-1.35
78,-15.1875,8.5375,5.425,8.3,10.6,0.65


### 2. Ковариационная матрица

In [4]:
cov_matrix = np.cov(data_centered, rowvar=False)
cov_matrix

array([[177.14161392,  44.40585443, 118.89082278, -15.90506329,
         15.27848101,  46.90822785],
       [ 44.40585443,  83.41629747,  78.65474684,  27.45696203,
         16.52151899,  20.83607595],
       [118.89082278,  78.65474684, 205.36139241,  19.7443038 ,
         37.65316456,  34.08734177],
       [-15.90506329,  27.45696203,  19.7443038 ,  47.55443038,
         25.99493671, -10.61518987],
       [ 15.27848101,  16.52151899,  37.65316456,  25.99493671,
         55.07848101,  18.37721519],
       [ 46.90822785,  20.83607595,  34.08734177, -10.61518987,
         18.37721519, 109.39493671]])

### 3. Собственные значения и собственные векторы
    

In [5]:
eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)
print('Собственные значения = ' ,eigenvalues,'\n')
print('Собственные векторы = ' ,eigenvectors)

Собственные значения =  [ 13.04851148  43.29783624  47.54565899  91.62374906 119.19843693
 363.23295919] 

Собственные векторы =  [[ 0.11985032 -0.48057759 -0.18402864  0.38369624  0.48113766 -0.58489168]
 [-0.38267687 -0.44473463  0.63460139 -0.19194274 -0.33942831 -0.31780026]
 [ 0.07171259  0.62146814 -0.01168077  0.12416326 -0.34038398 -0.69081144]
 [ 0.74643291 -0.36258469 -0.15389928 -0.22000562 -0.48702893 -0.04566848]
 [-0.49594227 -0.17519795 -0.72452153 -0.35322081 -0.22724366 -0.14836063]
 [ 0.17587151  0.15123381  0.12108531 -0.79205508  0.49859489 -0.2357706 ]]


### 4. Сортировка собственных значений и векторов

In [6]:
# Отсортируем по убыванию собственные значения
sorted_indices = np.argsort(eigenvalues)[::-1]
eigenvalues = eigenvalues[sorted_indices]

# И векторы
eigenvectors = eigenvectors[:, sorted_indices]

print('Собственные значения = ' ,eigenvalues,'\n')
print('Собственные векторы = ' ,eigenvectors)

Собственные значения =  [363.23295919 119.19843693  91.62374906  47.54565899  43.29783624
  13.04851148] 

Собственные векторы =  [[-0.58489168  0.48113766  0.38369624 -0.18402864 -0.48057759  0.11985032]
 [-0.31780026 -0.33942831 -0.19194274  0.63460139 -0.44473463 -0.38267687]
 [-0.69081144 -0.34038398  0.12416326 -0.01168077  0.62146814  0.07171259]
 [-0.04566848 -0.48702893 -0.22000562 -0.15389928 -0.36258469  0.74643291]
 [-0.14836063 -0.22724366 -0.35322081 -0.72452153 -0.17519795 -0.49594227]
 [-0.2357706   0.49859489 -0.79205508  0.12108531  0.15123381  0.17587151]]


### 5. Проекция данных

In [7]:
projected_data = np.dot(data_centered, eigenvectors)
projected_data

array([[-2.69832360e+01,  6.15374714e-01,  1.63825443e-01,
         4.35084026e+00,  9.60808957e+00,  7.45950418e-01],
       [-1.82561566e+01, -1.76341356e+00, -1.53145527e+00,
        -3.22047344e+00, -3.54168322e+00, -6.04110777e+00],
       [-6.82587003e+00,  1.57070429e+01, -7.43209725e+00,
         3.89442707e+00,  2.99626006e+00,  2.26236217e+00],
       [-3.01583714e+00,  2.23216396e+01,  3.97366303e+00,
        -1.41530264e+00,  7.81946878e+00, -5.99492674e+00],
       [-8.11676515e+00,  1.62255194e+01, -6.48310804e+00,
        -7.32102359e+00, -4.81288672e+00,  7.14756170e-01],
       [-1.86872208e+01,  9.91811710e+00, -5.68191267e+00,
        -6.27679253e+00, -1.33631682e+01, -2.39048103e+00],
       [-2.52283917e+01,  5.86358434e+00, -4.51031597e+00,
        -7.65427989e+00,  8.93447929e+00, -1.24215390e+00],
       [-2.46712148e+01, -2.04509038e-01, -1.57591966e+00,
        -9.48458339e+00,  7.24623759e+00, -1.48078338e+00],
       [ 7.46486617e+00,  1.99594493e+01,  6.202

### 6. Результаты

In [8]:
results = {
        "explained_variance": eigenvalues,
        "explained_variance_ratio": eigenvalues / np.sum(eigenvalues),
        "components": eigenvectors,
        "projected_data": projected_data
    }

print("Объяснённая дисперсия:", results['explained_variance'],'\n' )
print("Доля объяснённой дисперсии:", results['explained_variance_ratio'],'\n')
print("Главные компоненты (векторы):\n", results['components'],'\n')
print("Проекция данных (первые строки):\n", results['projected_data'][:5],'\n')

Объяснённая дисперсия: [363.23295919 119.19843693  91.62374906  47.54565899  43.29783624
  13.04851148] 

Доля объяснённой дисперсии: [0.53578359 0.17582261 0.13514881 0.07013181 0.06386609 0.01924709] 

Главные компоненты (векторы):
 [[-0.58489168  0.48113766  0.38369624 -0.18402864 -0.48057759  0.11985032]
 [-0.31780026 -0.33942831 -0.19194274  0.63460139 -0.44473463 -0.38267687]
 [-0.69081144 -0.34038398  0.12416326 -0.01168077  0.62146814  0.07171259]
 [-0.04566848 -0.48702893 -0.22000562 -0.15389928 -0.36258469  0.74643291]
 [-0.14836063 -0.22724366 -0.35322081 -0.72452153 -0.17519795 -0.49594227]
 [-0.2357706   0.49859489 -0.79205508  0.12108531  0.15123381  0.17587151]] 

Проекция данных (первые строки):
 [[-26.98323599   0.61537471   0.16382544   4.35084026   9.60808957
    0.74595042]
 [-18.25615656  -1.76341356  -1.53145527  -3.22047344  -3.54168322
   -6.04110777]
 [ -6.82587003  15.70704291  -7.43209725   3.89442707   2.99626006
    2.26236217]
 [ -3.01583714  22.32163963  

## Выведем полный алгоритм функцией

In [9]:
def pca_manual(data, n_components=None, print_results=False):
    """
    Реализация PCA вручную.
    
    :param data: DataFrame или массив данных (каждая строка — наблюдение, столбец — признак).
    :param n_components: Число главных компонент (если None, используются все).
    :return: Словарь с результатами PCA.
    """
    import numpy as np
    import pandas as pd
    
    def fix_signs(vectors):
        for i in range(vectors.shape[1]):
            # Если первый элемент отрицательный, инвертировать знак всего вектора
            if vectors[0, i] < 0:
                vectors[:, i] *= -1
        return vectors
    # 1. Центрирование данных
    data_centered = data - np.mean(data, axis=0)
    
    # 2. Ковариационная матрица
    cov_matrix = np.cov(data_centered, rowvar=False)
    
    # 3. Собственные значения и собственные векторы
    eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)
    eigenvectors=fix_signs(eigenvectors)
    # 4. Сортировка собственных значений и векторов
    sorted_indices = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[sorted_indices]
    eigenvectors = eigenvectors[:, sorted_indices]
    
    # 5. Выбор числа компонент
    if n_components is not None:
        eigenvectors = eigenvectors[:, :n_components]
        eigenvalues = eigenvalues[:n_components]
    
    # 6. Проекция данных
    projected_data = np.dot(data_centered, eigenvectors)
    
    loadings = pd.DataFrame(
        eigenvectors.transpose(),
        columns=[f'x{i+1}' for i in range(data.shape[1]-1)] + ['PCA_score'],
        index=[f'PC{i+1}' for i in range(len(eigenvectors.transpose()))]
    )
    
    # 7. Результаты
    results = {
        "explained_variance": eigenvalues,
        "explained_variance_ratio": eigenvalues / np.sum(eigenvalues),
        "components": eigenvectors,
        "projected_data": projected_data,
        'loadings':loadings
    }
    if print_results:
        # Вывод результатов
        print("Объяснённая дисперсия:", results['explained_variance'],'\n' )
        print("Доля объяснённой дисперсии:", results['explained_variance_ratio'],'\n')
        print("Главные компоненты (векторы):\n", results['components'],'\n')
        print("Проекция данных (первые строки):\n", results['projected_data'],'\n')
    
    return results

# Применение PCA
results = pca_manual(df3_1.iloc[:,1:].values, n_components=6)
results

{'explained_variance': array([363.23295919, 119.19843693,  91.62374906,  47.54565899,
         43.29783624,  13.04851148]),
 'explained_variance_ratio': array([0.53578359, 0.17582261, 0.13514881, 0.07013181, 0.06386609,
        0.01924709]),
 'components': array([[ 0.58489168,  0.48113766,  0.38369624,  0.18402864,  0.48057759,
          0.11985032],
        [ 0.31780026, -0.33942831, -0.19194274, -0.63460139,  0.44473463,
         -0.38267687],
        [ 0.69081144, -0.34038398,  0.12416326,  0.01168077, -0.62146814,
          0.07171259],
        [ 0.04566848, -0.48702893, -0.22000562,  0.15389928,  0.36258469,
          0.74643291],
        [ 0.14836063, -0.22724366, -0.35322081,  0.72452153,  0.17519795,
         -0.49594227],
        [ 0.2357706 ,  0.49859489, -0.79205508, -0.12108531, -0.15123381,
          0.17587151]]),
 'projected_data': array([[ 2.69832360e+01,  6.15374714e-01,  1.63825443e-01,
         -4.35084026e+00, -9.60808957e+00,  7.45950418e-01],
        [ 1.82561566e

In [10]:
# Применение PCA
results = pca_manual(df3_1.iloc[:,1:].values)
# Используем первую компоненту (или взвешенную сумму нескольких, если требуется)
df3_1['PCA_Score'] = results['projected_data'][:, 0]  # Первая главная компонента как общий индекс

# Ранжирование стран
data_sorted = df3_1.sort_values(by='PCA_Score', ascending=False)

# Вывод результатов
print("Доля объясненной дисперсии (вклад каждой компоненты):")
print(results['explained_variance_ratio'])

print("\nФакторные нагрузки (вклад признаков в компоненты):")
print(results['loadings'])

print("\nРанжированные страны по удовлетворённости:")
data_sorted[['Страна', 'PCA_Score']]

Доля объясненной дисперсии (вклад каждой компоненты):
[0.53578359 0.17582261 0.13514881 0.07013181 0.06386609 0.01924709]

Факторные нагрузки (вклад признаков в компоненты):
           x1        x2        x3        x4        x5  PCA_score
PC1  0.584892  0.317800  0.690811  0.045668  0.148361   0.235771
PC2  0.481138 -0.339428 -0.340384 -0.487029 -0.227244   0.498595
PC3  0.383696 -0.191943  0.124163 -0.220006 -0.353221  -0.792055
PC4  0.184029 -0.634601  0.011681  0.153899  0.724522  -0.121085
PC5  0.480578  0.444735 -0.621468  0.362585  0.175198  -0.151234
PC6  0.119850 -0.382677  0.071713  0.746433 -0.495942   0.175872

Ранжированные страны по удовлетворённости:


,Страна,PCA_Score
№,,
35,Парагвай,29.049238
41,Гондурас,26.991538
1,Израиль,26.983236
34,Боливия,26.558810
33,Таиланд,26.294956
...,...,...
65,Руанда,-31.830723
24,Грузия,-33.447196
73,Мали,-34.558369


In [11]:
data_sorted[['Страна', 'PCA_Score']].to_excel('ranged_countries.xlsx')

# Задача 3.2

Провести факторный анализ социальноэкономического развития регионов Российской Федерации по следующим показателям:  
- х1 – коэффициент Джини (индекс концентрации доходов), коэффициент; 
- х2  - коэффициент фондов (соотношение доходов 10% наиболее и 10% наименее обеспеченного населения), коэффициент; 
- х3 - доля населения с денежными доходами ниже величины прожиточного минимума, %; 
- х4 - среднедушевые денежные доходы, тыс. руб.; 
- х5 - дефицит дохода, %; 
- х6 - число родившихся в расчете на 1000 населения за год, %0; 
- х7 - число умерших в расчете на 1000 населения за год, %0; 
- х8 - демографическая нагрузка на население трудоспособного возраста, на 1000 человек трудоспособного возраста приходится лиц нетрудоспособных возрастов на начало года, %0; 
- х9 - уровень безработицы по методологии МОТ, %; 
- х10  - валовый региональный продукт на душу населения, тыс. руб.; 
- х11 - инвестиции в основной капитал на душу населения, тыс. руб. 

Данные для анализа представлены в следующей таблице. Распределение показателей по вариантам выполнения задания: 
- Вариант 1 - х1; х3; х4; х6; х8; х9; х10; 
- Вариант 2 - х1; х3; х4; х7; х8; х9; х11; 
- Вариант 3 - х2; х4; х5; х6; х8; х9; х10; 
- Вариант 4 - х2; х4; х5; х7; х8; х9; х11. 

В моем случае:

- Вариант 3 - х2; х4; х5; х6; х8; х9; х10; 

## Возьмем данные из `Исходные данные для задачи 3.2.pdf`

In [12]:
text = '''Регион Показатели х1 х2 х3 х4 х5 х6 х7 х8 х9 х10 х11 Белгородская область 0,40 14,30 10,20 14,00 0,90 11,00 14,40 605,33 4,80 209,62 67,85 Брянская область 0,39 12,80 15,30 11,39 1,70 11,10 16,90 628,60 10,70 97,40 18,80 Владимирская область 0,35 10,10 20,40 10,47 2,80 10,80 18,40 624,83 8,80 122,01 32,41 Воронежская область 0,40 14,60 21,20 11,78 2,80 10,40 17,00 640,78 8,60 127,16 40,50 Ивановская область 0,35 10,00 23,40 9,15 3,50 10,60 18,60 630,14 10,80 79,98 22,73 Калужская область 0,39 12,80 12,40 13,37 1,20 10,50 16,70 617,00 6,20 152,60 54,48 Костромская область 0,36 10,60 19,80 10,39 2,70 11,80 17,20 609,74 8,30 116,74 22,81 Курская область 0,38 12,10 12,00 12,62 1,20 10,80 17,60 638,83 8,80 144,95 38,91 Липецкая область 0,39 12,80 10,20 14,25 0,90 10,90 16,50 625,55 5,60 225,69 69,89 Московская область 0,41 15,40 10,10 20,82 0,90 11,00 15,90 584,20 4,90 251,84 66,49 Орловская область 0,40 13,90 18,60 10,46 2,30 10,40 16,60 625,97 9,90 116,47 32,76 Рязанская область 0,37 11,10 16,40 11,70 2,00 10,10 17,80 663,39 9,20 128,93 46,52 Смоленская область 0,38 12,20 14,80 12,88 1,70 10,80 19,20 595,65 7,70 124,21 37,84 Тамбовская область 0,40 14,60 11,80 12,01 1,10 9,30 17,30 654,77 9,10 111,87 37,67 Тверская область 0,35 10,00 14,30 12,09 1,60 11,10 20,00 658,15 7,70 143,29 36,28 Тульская область 0,37 11,70 12,00 13,53 1,20 9,50 19,40 666,38 6,10 147,85 30,35 Ярославская область 0,38 12,20 16,20 13,00 1,90 10,90 16,60 623,34 8,00 167,55 38,20 г.Москва 0,53 35,40 10,00 44,49 0,70 11,00 11,40 548,17 2,70 804,72 86,96 Республика Карелия 0,36 10,40 17,10 13,51 2,10 11,50 15,50 549,43 10,00 170,11 33,02 Республика Коми 0,43 17,20 16,80 19,99 1,80 12,40 12,80 482,27 11,80 306,86 85,30 Архангельская область 0,38 12,40 14,70 16,56 1,60 12,30 14,40 556,97 7,20 235,30 103,83 Вологодская область 0,37 11,40 18,60 12,06 2,40 12,40 16,20 585,10 7,90 244,25 59,16 46  Калининградская область 0,38 12,00 13,50 14,50 1,40 11,50 14,60 547,76 10,90 193,86 71,52 Ленинградская область 0,37 11,50 14,80 12,14 1,70 9,20 17,00 574,19 7,20 235,14 98,78 Мурманская область 0,39 13,50 14,80 21,00 1,60 10,80 12,00 451,14 7,60 255,01 53,83 Новгородская область 0,40 14,20 17,00 13,07 2,00 11,20 20,40 639,47 6,40 177,88 50,40 Псковская область 0,37 11,70 16,80 11,21 2,00 10,40 20,90 642,46 11,00 104,80 23,39 г.Санкт-Петербург 0,44 19,30 9,60 21,33 0,70 11,30 14,10 592,91 4,10 310,57 80,21 Республика Адыгея 0,38 12,40 19,30 10,20 2,50 12,40 14,00 646,04 7,90 84,25 25,34 Республика Калмыкия 0,37 11,80 36,30 6,98 7,70 15,10 11,00 559,30 16,60 73,01 27,51 Краснодарский край 0,41 15,70 18,60 13,71 2,20 12,40 13,70 634,25 7,20 157,59 62,69 Астраханская область 0,40 13,90 15,40 12,82 1,70 14,20 13,40 571,51 10,00 146,61 65,03 Волгоградская область 0,37 11,40 13,00 12,52 1,40 11,40 14,50 614,76 10,40 165,81 31,74 Ростовская область 0,40 13,80 16,00 12,79 1,80 10,90 14,70 610,99 8,40 135,68 41,53 Республика Дагестан 0,40 13,80 9,20 13,80 0,80 18,50 6,10 594,02 13,20 78,25 31,96 Республика Ингушетия 0,33 8,70 36,10 6,41 7,60 18,70 3,70 620,40 52,90 38,11 10,77 Кабардино-Балкарская  Республика 0,37 11,80 16,20 9,61 1,90 13,60 9,40 551,90 14,40 65,74 15,60 Карачаево-Черкесская  Республика 0,38 12,00 16,20 10,14 1,90 14,50 11,00 608,27 12,20 82,51 25,62 Республика Северная  Осетия - Алания 0,38 12,50 11,10 11,84 1,10 14,30 11,40 645,41 10,60 82,42 23,28 Ставропольский край 0,39 12,90 19,40 11,42 2,50 12,10 13,20 597,65 8,70 101,65 25,91 Республика Башкортостан 0,44 18,60 11,20 16,12 0,90 13,70 13,10 587,20 9,30 184,83 51,08 Республика Марий Эл 0,39 13,10 24,40 9,14 3,70 12,70 14,90 541,48 11,60 96,06 29,93 Республика Мордовия 0,37 11,70 19,70 9,57 2,60 9,80 15,70 581,81 5,40 113,04 45,33 Республика Татарстан 0,41 15,40 8,60 15,52 0,70 12,40 12,70 584,01 8,50 245,16 71,34 Удмуртская Республика 0,37 11,20 15,30 10,87 1,80 13,80 13,20 557,22 8,70 157,01 32,59 Чувашская Республика 0,36 11,00 19,40 9,46 2,60 12,60 13,70 568,77 11,50 123,05 38,73 47  Пермский край 0,43 17,60 15,30 17,12 1,60 13,30 15,00 579,33 10,10 224,53 56,16 Кировская область 0,37 11,20 18,30 10,96 2,30 11,40 16,60 584,65 10,90 107,22 28,79 Нижегородская область 0,39 13,20 13,50 14,40 1,40 10,90 17,50 615,77 7,40 178,49 58,40 Оренбургская область 0,38 12,70 16,20 11,48 1,90 13,30 13,80 576,60 8,80 200,94 48,79 Пензенская область 0,38 12,50 15,10 11,50 1,70 10,30 15,80 620,94 7,90 104,87 39,91 Самарская область 0,45 20,60 16,10 18,05 1,60 11,50 14,80 582,24 6,10 222,73 45,96 Саратовская область 0,38 11,90 19,40 10,38 2,50 10,70 15,00 601,61 8,80 128,21 30,33 Ульяновская область 0,39 13,40 20,40 10,48 2,70 10,50 14,90 576,43 9,00 115,49 38,13 Курганская область 0,41 15,50 17,20 12,61 2,00 12,60 15,70 624,57 13,30 111,28 33,59 Свердловская область 0,43 17,90 11,00 19,24 0,90 12,80 14,30 577,47 8,30 214,88 54,75 Тюменская область 0,46 21,60 12,20 27,68 1,00 15,50 8,80 446,75 6,80 928,37 298,91 Челябинская область 0,40 14,10 11,20 14,94 1,00 13,00 14,30 590,58 8,00 189,46 50,83 Республика Алтай 0,34 9,40 32,60 10,72 6,30 20,30 11,90 591,08 13,90 90,23 34,96 Республика Бурятия 0,41 15,20 19,70 12,66 2,50 17,40 13,00 556,86 14,10 132,07 24,77 Республика Тыва 0,37 11,60 29,10 9,47 5,10 26,10 11,60 612,28 21,50 78,04 10,43 Республика Хакасия 0,38 12,00 19,10 11,36 2,50 15,00 13,50 555,99 9,00 137,80 19,15 Алтайский край 0,36 10,70 25,30 9,61 4,00 12,40 14,30 574,24 12,30 107,42 22,24 Забайкальский край 0,40 13,90 20,10 12,55 2,60 15,90 13,70 558,59 12,20 126,00 42,10 Красноярский край 0,43 17,10 19,20 16,08 2,30 13,20 13,00 535,19 9,50 256,13 63,88 Иркутская область 0,42 16,00 19,10 13,62 2,30 15,20 13,90 567,86 10,80 182,20 51,01 Кемеровская область 0,40 14,30 12,20 13,39 1,20 13,30 15,90 570,48 9,70 204,04 52,89 Новосибирская область 0,41 15,20 16,60 14,92 1,90 12,90 14,10 569,43 10,00 174,42 46,44 Омская область 0,41 14,80 15,30 13,94 1,60 12,70 13,60 555,74 8,50 175,06 42,20 Томская область 0,39 12,80 17,80 13,77 2,20 13,20 12,90 506,21 8,70 242,80 75,79 Республика Саха  0,40 14,40 19,50 21,27 2,50 16,80 9,80 534,61 8,70 320,84 162,20 Камчатский край 0,37 11,90 23,10 22,64 3,40 11,90 11,80 462,03 7,70 227,07 46,18 Приморский край 0,39 12,80 19,40 15,14 2,50 11,80 13,80 542,44 9,80 160,33 36,52 48  Хабаровский край 0,39 13,00 18,80 19,17 2,40 12,50 13,60 521,75 10,50 194,76 55,16 Амурская область 0,37 11,50 23,10 13,12 3,40 13,20 14,60 539,97 8,70 157,76 74,74 Магаданская область 0,40 14,60 16,30 23,69 1,80 12,10 13,30 455,89 6,60 255,17 72,88 Сахалинская область 0,42 15,90 12,00 27,55 1,10 12,10 14,50 506,55 10,00 650,26 291,14 Еврейская Автономная  область 0,38 12,00 22,50 12,97 3,20 13,20 14,80 542,15 8,50 143,93 43,06 Чукотский Автономный  округ 0,41 15,30 9,30 39,18 0,80 14,20 13,00 434,73 4,40 615,31 176,28 '''

# Уберем номера страниц, заменим запятые на точки, а из строки сделаем список
text_split = text.replace(',','.').replace(' 46 ','').replace(' 44 ','').replace(' 45 ','').replace(' 47 ','').replace(' 48 ','').split(' ')

# В полученном списке преобразуем значения с плавающей точкой
table = []
for i in range(len(text_split)):
    try:
        table.append(float(text_split[i]))
    except:
        table.append(text_split[i])

# Зададим временные переменные
data = []
temp = []
k = 0
word = ''

# Пройдемся по всему списку
for i in range(len(table)):
    # Отсекаем слово показатели
    if table[i]=='Показатели':
        continue
    
    # Добавляем заголовки
    if i in list(range(13)):
        temp.append(table[i])
        continue
    if i == 13:
        data.append(temp)
        temp = []
    
    # Проверяем, строка это или нет
    if isinstance(table[i],str):
        # Если строка, то добавляем к слову
        word+= ' ' + table[i]
        continue
    
    elif isinstance(table[i],float):
        # Если не строка, то добавляем слово во временный список, а потом убираем
        if isinstance(table[i-1],str):
            temp.append(word)
            word = ''
            k+=1
        temp.append(table[i])
        k+=1
       
    # Так как в каждой строке 12 значений
    # в двумерный список будем добавлять каждые 12 значений    
    if k!=0 and k%12==0:
        data.append(temp)
        temp = []
    

df3_2 = pd.DataFrame(np.array(data))
# Обозначим первую строку как заголовок, первый столбец за индекс и выполним обратное преобразование названий стран
df3_2.columns = df3_2.iloc[0]
df3_2 = df3_2.drop(0).reset_index(drop=True)
df3_2.columns = ['Регион']+[f'x{i+1}' for i in range(11)]
df3_2

,Регион,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11
0,Белгородская область,0.4,14.3,10.2,14.0,0.9,11.0,14.4,605.33,4.8,209.62,67.85
1,Брянская область,0.39,12.8,15.3,11.39,1.7,11.1,16.9,628.6,10.7,97.4,18.8
2,Владимирская область,0.35,10.1,20.4,10.47,2.8,10.8,18.4,624.83,8.8,122.01,32.41
3,Воронежская область,0.4,14.6,21.2,11.78,2.8,10.4,17.0,640.78,8.6,127.16,40.5
4,Ивановская область,0.35,10.0,23.4,9.15,3.5,10.6,18.6,630.14,10.8,79.98,22.73
...,...,...,...,...,...,...,...,...,...,...,...,...
74,Амурская область,0.37,11.5,23.1,13.12,3.4,13.2,14.6,539.97,8.7,157.76,74.74
75,Магаданская область,0.4,14.6,16.3,23.69,1.8,12.1,13.3,455.89,6.6,255.17,72.88
76,Сахалинская область,0.42,15.9,12.0,27.55,1.1,12.1,14.5,506.55,10.0,650.26,291.14
77,Еврейская Автономная область,0.38,12.0,22.5,12.97,3.2,13.2,14.8,542.15,8.5,143.93,43.06


- Вариант 3 - х2; х4; х5; х6; х8; х9; х10; 

In [13]:
df3_2_var_3 = df3_2[['Регион', 'x2','x4','x5','x6','x8','x9','x10']].copy()

for i in df3_2_var_3.columns[1:]:
    df3_2_var_3[i]=df3_2_var_3[i].transform(float)
    
df3_2_var_3

,Регион,x2,x4,x5,x6,x8,x9,x10
0,Белгородская область,14.3,14.00,0.9,11.0,605.33,4.8,209.62
1,Брянская область,12.8,11.39,1.7,11.1,628.60,10.7,97.40
2,Владимирская область,10.1,10.47,2.8,10.8,624.83,8.8,122.01
3,Воронежская область,14.6,11.78,2.8,10.4,640.78,8.6,127.16
4,Ивановская область,10.0,9.15,3.5,10.6,630.14,10.8,79.98
...,...,...,...,...,...,...,...,...
74,Амурская область,11.5,13.12,3.4,13.2,539.97,8.7,157.76
75,Магаданская область,14.6,23.69,1.8,12.1,455.89,6.6,255.17
76,Сахалинская область,15.9,27.55,1.1,12.1,506.55,10.0,650.26
77,Еврейская Автономная область,12.0,12.97,3.2,13.2,542.15,8.5,143.93


In [14]:
# Применение PCA
results = pca_manual(df3_2_var_3.iloc[:,1:].values,7)

# Используем первую компоненту (или взвешенную сумму нескольких, если требуется)
df3_2_var_3['PCA_Score'] = results['projected_data'][:, 0]  # Первая главная компонента как общий индекс

# Ранжирование стран
data_sorted = df3_2_var_3.sort_values(by='PCA_Score', ascending=False)

# Вывод результатов
print("Доля объясненной дисперсии (вклад каждой компоненты):")
print(results['explained_variance_ratio'])

print("\nФакторные нагрузки (вклад признаков в компоненты):")
print(results['loadings'])
data_sorted[['Регион', 'PCA_Score']].to_excel('ranged_regions.xlsx')
data_sorted[['Регион', 'PCA_Score']]

Доля объясненной дисперсии (вклад каждой компоненты):
[9.26088181e-01 7.18164959e-02 1.33235626e-03 4.08136189e-04
 1.80205529e-04 1.43967129e-04 3.06579631e-05]

Факторные нагрузки (вклад признаков в компоненты):
           x1        x2        x3        x4        x5        x6  PCA_score
PC1  0.015968  0.035834 -0.003201 -0.000293 -0.210680 -0.011864   0.976690
PC2  0.016506 -0.007660 -0.008392 -0.016211  0.977118 -0.015468   0.210563
PC3  0.073979  0.158361 -0.137684 -0.285273 -0.015870 -0.931859  -0.022299
PC4  0.592966  0.779150 -0.042090  0.129238  0.007726  0.146844  -0.034930
PC5  0.350346 -0.344860  0.053958  0.819433  0.000025 -0.289712   0.003834
PC6  0.720007 -0.493762 -0.025452 -0.471008 -0.022723  0.121525   0.002694
PC7  0.035186  0.061443  0.987741 -0.091298  0.005150 -0.104849   0.000217


,Регион,PCA_Score
56,Тюменская область,750.018358
17,г.Москва,608.757108
76,Сахалинская область,465.659411
78,Чукотский Автономный округ,447.129065
70,Республика Саха,137.767004
...,...,...
60,Республика Тыва,-116.367351
28,Республика Адыгея,-117.202040
4,Ивановская область,-118.135724
38,Республика Северная Осетия - Алания,-118.824398


Метод главных компонент (PCA) и сингулярное разложение (SVD) используют линейные алгебраические преобразования для перехода от исходного пространства признаков к новому пространству главных компонент. Вот подробное описание процесса преобразования:

---

### Этапы преобразования данных:

#### 1. **Центрирование данных**
Перед применением PCA/SVD данные центрируются, чтобы каждая переменная имела среднее значение 0.  
Это устраняет смещение и делает анализ фокусированным на вариации данных:  
$$
X_{\text{centered}} = X - \mu,
$$
где $ \mu $ — вектор средних значений признаков.

---

#### 2. **Ковариационная матрица (для PCA)**
Для PCA рассчитывается ковариационная матрица, которая отражает совместную изменчивость признаков:
$$
\text{Cov}(X) = \frac{1}{n-1} X_{\text{centered}}^\top X_{\text{centered}}
$$

Элементы ковариационной матрицы показывают степень линейной зависимости между парами признаков.

---

#### 3. **Собственные значения и собственные векторы (для PCA)**
Решается задача нахождения собственных значений и собственных векторов ковариационной матрицы:
$$
\text{Cov}(X) \cdot v = \lambda \cdot v,
$$
где:
- $ v $ — собственные векторы (направления новых осей — главных компонент),
- $ \lambda $ — собственные значения (дисперсия вдоль соответствующих компонент).

Собственные векторы упорядочиваются по убыванию собственных значений. Первые несколько собственных векторов с наибольшими $ \lambda $ используются для создания нового пространства признаков.

---

#### 4. **Сингулярное разложение (для SVD)**
Для вычисления PCA часто используется сингулярное разложение (SVD), так как это более численно устойчивый метод.  
Матрица $ X_{\text{centered}} $ представляется в виде:
$$
X_{\text{centered}} = U \cdot \Sigma \cdot V^\top,
$$
где:
- $ U $ — матрица левых сингулярных векторов (проекции данных на новое пространство),
- $ \Sigma $ — диагональная матрица сингулярных значений (корни собственных значений),
- $ V^\top $ — транспонированная матрица правых сингулярных векторов (направления главных компонент, аналог собственных векторов).

---

#### 5. **Проекция данных на новое пространство**
После нахождения главных компонент данные проецируются на новое пространство.  
Проекция осуществляется так:
$$
Z = X_{\text{centered}} \cdot V,
$$
где:
- $ Z $ — данные в пространстве главных компонент,
- $ V $ — собственные векторы (или правые сингулярные векторы из SVD).

---

### Итоговые преобразования:

- Каждая главная компонента представляет собой линейную комбинацию исходных признаков:
$$
PC_i = \sum_{j=1}^m w_{ij} \cdot x_j,
$$
где $ w_{ij} $ — элементы $ i $-го собственного вектора, а $ x_j $ — стандартизированные значения признаков.

- Главные компоненты упорядочены так, что первая объясняет наибольшую долю дисперсии, вторая — вторую по величине, и так далее.